## Nikolaos Papadopoulos

#### Course: Numerical Optimization and Large Scale Linear Algebra

#### 1st assignment: PageRank

In this assignment we were tasked with creating a page-rank algorithm given a .txt with the nodes and their out-links(No weights) in a condensed form.

First,we prepare our data so as to create the sparse matrix that is going to represent all the nodes and their links.

In [1]:
#Necessary imports
import os
import numpy as np
import time
import pandas as pd
from  scipy import sparse
from scipy.sparse import csc_matrix
from scipy.stats import rankdata

In [2]:
#Setting my directory
os.chdir('.\\NumericalOptimization\\Assignment 1')

In [3]:
#Reading the file and skipping the first 2 rows as they are not needed
data = pd.read_csv('out.web-BerkStan',sep=' ',header=None,skiprows=2)
#Extra column was created(Filled with NAN) so I dropped it
data=data.drop(columns=[2])
#Nodes in python start at 0 but our text starts with node number 1 so we are subtracting 1 from every node
data[0]=data[0]-1
data[1]=data[1]-1
rows = (data[0]) 
cols = (data[1])
#Matrix N
n = len(set(rows) | set(cols))

In [4]:
#Tidying the dataframe
data=data.rename(columns = {0:'Node',1:'Outlink'})

In [5]:
#Outlinks per node
weight=data.Node.value_counts() 

In [6]:
#Getting the probability Distribution of each node
data['weight'] = data['Node'].apply(lambda r : 1/weight[r])

In [7]:
#Making sure weight has been distributed correctly among nodes
data[:10]

,Node,Outlink,weight
0,0,1,0.111111
1,0,2,0.111111
2,0,3,0.111111
3,0,4,0.111111
4,0,5,0.111111
5,0,6,0.111111
6,0,7,0.111111
7,0,8,0.111111
8,0,9,0.111111
9,8,10,0.010417


## 1) Creating the Sparse Matrix

In [8]:
#Sparse matrix
matrix = sparse.csr_matrix((data.weight,(rows, cols)))

In [9]:
#Making sure the dimensions are correct
matrix.shape

(685230, 685230)

In [10]:
#Creating a as instructed
a = np.zeros(n)
a[list(set(cols).difference(set(rows)))] = 1
np.count_nonzero(a)

4744

In [11]:
#Creating E,V as instructed on the pagerank paper for the power method
#(Am using the transposed version so as not to transpose when calling the function to save time)
e = np.ones((matrix.shape[0],1))
v = np.ones((matrix.shape[0],1))/matrix.shape[0] 

## Defining the Power Method algorithm

In [12]:
def power_method(P,a,alpha,v,tol):
    #Creating C so as to initialize the while loop.
    c = 1
    #Initiliazing x_previous
    x_previous = e/len(a)
    #Iteration counter
    iteration = 0
    #Termination criterion
    while c>tol:
        iteration = iteration+1
        #Getting the probability vector
        xkt = alpha*sparse.csr_matrix.dot(x_previous.T,P)+(alpha*sparse.csr_matrix.dot(x_previous.T,a)+ (1-alpha))*v.T
        #Getting the transposed form
        xk = xkt.T
        # Euclidean Norm
        c  = np.linalg.norm(xk - x_previous,2)
        #Updating x_previous
        x_previous = xk
    print (iteration,'iterations')
    return (x_previous)

In [13]:
#Setting up the timer
start = time.time()
b = power_method(matrix,a,0.85,v,1e-8)
end = time.time()
print('Power method with a=0.85 converges in ',round((end-start),3),'seconds')

71 iterations
Power method with a=0.85 converges in  2.23 seconds


## 2) Changing Alpha value and findings

In [14]:
#Increasing alpha value to 0.99
start = time.time()
c = power_method(matrix,a,0.99,v,1e-8)
end = time.time()
print('Power method with a=0.99 converges in ',round((end-start),3),'seconds')

1217 iterations
Power method with a=0.99 converges in  36.801 seconds


In [15]:
c = power_method(matrix,a,0.99,v,1e-8)

1217 iterations


We see that for a=0.85 we have iterated 71 times and converged in 2 seconds whereas as we increase a to 0.99 we get 1217 iterations and the convergence time is increased to 34 seconds.

As expected,the smaller the alpha is, the faster the convergence, but the smaller α is, the less the true hyperlink structure of the web is used to determine webpage importance. And slightly different values for α can produce very different PageRanks. Moreover, as α → 1, not only does convergence slow drastically, but sensitivity issues begin to surface as
well.

However, in addition to the computational reasons for choosing
α = .85, this choice for α also carries some intuitive weight: α = .85 implies that roughly 5/6 of the
time a Web surfer randomly clicks on hyperlinks (i.e., following the structure of the Web, as captured
by the α  ̄P part of the formula), while 1/6 of the time this Web surfer will go to the URL line and type
the address of a new page to “teleport” to (as captured by the (1 − α)evT part of the formula).

Whereas α = .99, not only slows convergence of the power method, but also
places much greater emphasis on the hyperlink structure of the Web and much less on the teleportation
tendencies of surfers.

The PageRank vector derived from α = .99 can be vastly different from that obtained using α = .85.
Perhaps it gives a “truer” PageRanking. Experiments with various α’s show significant variation in
rankings produced by different values of α [97, 98, 110]. As expected, the top section of the ranking
changes only slightly, yet as we proceed down the ranked list we see more and more variation.

We see that the ranking of the 100 highest ranking nodes is different using a= 0.85 and a=0.99

In [16]:
rankdata(b)[0:100]==rankdata(c)[0:100]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [17]:
#Choosing method ordinal as all values are given a distinct rank.
pagerankb=rankdata(b,method='ordinal')
pagerankc=rankdata(c,method='ordinal')

In [18]:
#Node ranks
pagerankb[0:100]

array([680685, 662814, 650820, 668985, 646655, 663419, 663253, 661916,
       684761, 685229, 608685, 608686, 608687, 608688, 673309, 608689,
       608690, 608691, 608692, 608693, 608694, 683845, 611313, 608695,
       611314, 611315, 611316, 611317, 611318, 611319, 608696, 608697,
       620318, 608698, 620319, 620320, 675408, 620321, 620322, 620323,
       620324, 608699, 608700, 608701, 608702, 608703, 608704, 608705,
       608706, 608707, 608708, 608709, 608710, 608711, 683823, 683647,
       608712, 608713, 608714, 608715, 608716, 608717, 608718, 608719,
       608720, 608721, 608722, 608723, 608724, 608725, 608726, 608727,
       683711, 608728, 608729, 608730, 608731, 608732, 608733, 608734,
       608735, 608736, 608737, 608738, 683834, 608739, 608740, 608741,
       608742, 608743, 608744, 608745, 608746, 608747, 609954, 608748,
       608749, 683854, 608750, 608751], dtype=int64)

In [19]:
#Highest 100 node ranking for a = 0.85
pagerankb.argsort()[::-1][:100]

array([288238,      9, 225465,  54130,   1283, 564114, 288346,  65122,
          113,    114,  51590,    470,    476,    475,    504,    501,
          500,    497,    496,    495,    477,    474,    473,    472,
       288245,    597,    600,  54199,  54207,  51553,    499,    505,
          507,    498,    506,    502,  71439, 476639,    607,  51593,
        51588,  65160,  54197, 288967,  54201,  54187,  54195,  54194,
        54198, 483455,  54200,  54186, 482866,  54189, 288246,  54196,
          609,    608, 483446, 483468,  65159,    596, 289060,    589,
          651,  47428,  65161,    617, 184997,    615,    618,    625,
          624,  54138, 278503,    626,  53845,  55358, 508986, 508615,
        55927, 288256, 508999, 508977,   1791,   1801, 288243, 554442,
        55928,  55929, 660726, 483192, 660739, 660717, 554443, 220432,
        52196,    115,  70702,   1800], dtype=int64)

In [20]:
#Highest 100 node ranking for a = 0.99
pagerankc.argsort()[::-1][:100]

array([ 51553,   1283, 288238, 288730,      9,  47428,  54140,    470,
          476,    475,    504,    501,    500,    497,    496,    495,
          477,    474,    473,    472, 564114,    499,    505,    507,
          498,    506,    502,  71439, 288243,  53845,  54207,  54199,
         2120, 225465,  51590, 288346,  54130,  70583, 483455, 482866,
        70549,  54142,  54141,  54197, 483468, 483446,  54201,  54187,
       466840,  54195,  54194,  54198,  54200,  54186,  54189,  54196,
       119870, 466841,  51557,  55358, 278503, 466843, 225576,  55927,
       288729,    114, 466842,  53854, 508986, 508615, 199990, 199989,
       379621, 508999, 508977, 451663,  51566,   1088,   1087,  55928,
        55929, 660726, 483192,    113,  47441,  65610,  65614, 660739,
        65612,  65611, 660717,  65613,  65609, 152821,  51555, 332485,
        70702,  53837,  51593,  70698], dtype=int64)

## 3) Which nodes converge faster?

To answer this question I'm going to get the thousand highest ranking nodes and the thousand lowest ranking nodes in conjuction with a power method test function I'm going to check at which stage these nodes get close(given a certain threshold) to their final values.

The threshold will be the difference of the final page-rank value with the current page-rank value of all given nodes.
When that differece meets a threshold a message will be presented letting the user know.

In [21]:
#10 highest ranking nodes (Storing these values for later)
tenhighest = pagerankb.argsort()[::-1][:10]
#10 lowest ranking nodes
tenlowest = pagerankb.argsort()[::-1][-10:]

In [22]:
#Thousand highest ranking nodes
highestbunch = pagerankb.argsort()[::-1][:1000]
#Thousand lowest ranking nodes
lowestbunch = pagerankb.argsort()[::-1][-1000:]

In [23]:
def which_converge_first(P,a,alpha,v,tol):
    #As the previous power method function
    c = 1
    x_previous = e/len(a)
    iteration = 0
    t1=1
    t2=1
    #Termination criterion
    while c>tol:
        iteration = iteration+1
        xkt = alpha*sparse.csr_matrix.dot(x_previous.T,P)+(alpha*sparse.csr_matrix.dot(x_previous.T,a)+ (1-alpha))*v.T
        xk = xkt.T
        #Euclidian Norm
        c  = np.linalg.norm(xk - x_previous,2) 
        x_previous = xk
        if ((x_previous[pagerankb.argsort()[::-1][:1000]] - b[highestbunch]) < [1e-8]).all() and t1!=0:
            print('Iterated',iteration,'times for highest ranking nodes convergence')
            t1=0
        if ((x_previous[pagerankb.argsort()[::-1][-1000:]] - b[lowestbunch]) < [1e-8]).all() and t2!=0:
            print('Iterated',iteration,'time for lowest ranking nodes convergence')
            t2=0
            print(' ')

In [24]:
which_converge_first(matrix,a,0.85,v,1e-8)

Iterated 1 time for lowest ranking nodes convergence
 
Iterated 63 times for highest ranking nodes convergence


As we see from the results above it took the highest ranking nodes(More 'important' ones) 63 iterations to reach the threshold I've chosen but the lowest ranking nodes only needed 1 iteration as their value remained the lowest one(Which they were initialized with)

The Highest Ranking Nodes need on average more iterations to converge than the Lowest Ranking Nodes. This is to no surprise due to the fact that the low rank nodes tend to have more outgoing than incoming connections, thus "flowing" pagerank faster towards other nodes while their pagerank is fastly converging to a value near the predefined threshold. The opposite happens to the high rank nodes, where the vast incoming connections give boost to their pagerank constantly until a point where they fluctuate around the specific threshold.

High-rank nodes have their page-rank score boosted because of the incoming connections from mostly other important nodes until they reach a point where they bounce around their final value.

## 4) Adding a new node

In [25]:
data

,Node,Outlink,weight
0,0,1,0.111111
1,0,2,0.111111
2,0,3,0.111111
3,0,4,0.111111
4,0,5,0.111111
...,...,...,...
7600590,684759,684750,0.076923
7600591,684759,684756,0.076923
7600592,684759,684757,0.076923
7600593,684759,684758,0.076923


### Our new node will be numbered as 685230

And it will out-link to the 10 highest ranking nodes as well as have in-links from 500 amongst the lower half part of all the nodes.

In [26]:
#10 highest ranking nodes
x1=pagerankb.argsort()[::-1][:10]

In [27]:
#Nodes that were evaluated on the lowest half of the ranking
x2pre=pagerankb.argsort()[::-1][-342615:]

np.random.seed(seed=1)
#Picking 500 nodes randomly
x2=np.random.choice(x2pre,500,replace=False)

In [28]:
#Getting the wanted nodes-outlinks combos
x1new=np.stack(([685230]*10,x1),axis=1)
x2new=np.stack((x2,[685230]*500),axis=1)
#Combining them together
xfinal=np.vstack((x1new,x2new))

In [29]:
#Creating a new dataframe
xfinal = pd.DataFrame(xfinal)
xfinal=xfinal.rename(columns={0:'Node',1:'Outlink'})
#Stacking them together
finaldf=pd.concat([data,xfinal])
finaldf

,Node,Outlink,weight
0,0,1,0.111111
1,0,2,0.111111
2,0,3,0.111111
3,0,4,0.111111
4,0,5,0.111111
...,...,...,...
505,320548,685230,NaN
506,398397,685230,NaN
507,499291,685230,NaN
508,613076,685230,NaN


In [30]:
#Getting the probability Distribution of each node
weight2=finaldf.Node.value_counts()
finaldf['weight'] = finaldf['Node'].apply(lambda r : 1/weight2[r])

#Creating the new sparse matrix
rows = (finaldf['Node']) 
cols = (finaldf['Outlink'])
n = len(set(rows) | set(cols))
a = np.zeros(n)
a[list(set(cols).difference(set(rows)))] = 1


matrix2 = sparse.csr_matrix((finaldf.weight,(rows, cols)))

#Creating E,V
e = np.ones((matrix2.shape[0],1))
v = np.ones((matrix2.shape[0],1))/matrix2.shape[0] 

In [31]:
#Making sure shapes match
matrix2.shape

(685231, 685231)

In [32]:
b2 = power_method(matrix2,a,0.85,v,1e-8)

71 iterations


In [33]:
#10 highest ranking nodes
rankdata(b2,method='ordinal').argsort()[::-1][:10]

array([288238,      9, 225465,  54130,   1283, 564114, 288346,  65122,
          113,    114], dtype=int64)

In [34]:
rankdata(b2,method='ordinal')[:10]

array([680686, 662815, 650821, 668984, 646659, 663420, 663253, 661918,
       684762, 685230], dtype=int64)

In [35]:
rankdata(b2,method='ordinal')[-1]

680848

In [36]:
print('The new node is ranked at',685231-rankdata(b2,method='ordinal')[-1],'out of 685231 total nodes')

The new node is ranked at 4383 out of 685231 total nodes


In [37]:
#Checking for a = 0.99
c2 = power_method(matrix2,a,0.99,v,1e-8)

1217 iterations


## 5) Adding a new node v2

This time,the 10 highest ranking nodes will point(out-link) to our node and our node will point to the 10 lowest ranking nodes.

In [38]:
#Inversing in-links with out-links from the previous question
x1newnew=x1new[:,[1,0]]
x1newnew

array([[288238, 685230],
       [     9, 685230],
       [225465, 685230],
       [ 54130, 685230],
       [  1283, 685230],
       [564114, 685230],
       [288346, 685230],
       [ 65122, 685230],
       [   113, 685230],
       [   114, 685230]], dtype=int64)

In [39]:
#10 lowest ranking nodes
x2testing=pagerankb.argsort()[::-1][-10:]
#Out-linking to the 10 lowest ranking nodes
x2newnew=np.stack(([685230]*10,x2testing),axis=1)
x2newnew

array([[685230,   3558],
       [685230,   3557],
       [685230,   3368],
       [685230,   3367],
       [685230,   3226],
       [685230,   2860],
       [685230,    781],
       [685230,    780],
       [685230,    779],
       [685230,    778]], dtype=int64)

In [40]:
#Stacking them together
xfinal2=np.vstack((x1newnew,x2newnew))
xfinal2 = pd.DataFrame(xfinal2)
xfinal2=xfinal2.rename(columns={0:'Node',1:'Outlink'})
#Creating the new final dataframe
finaldf2=pd.concat([data,xfinal2])

In [41]:
#Getting the probability Distribution of each node
weight3=finaldf2.Node.value_counts()
finaldf2['weight'] = finaldf2['Node'].apply(lambda r : 1/weight3[r])

#Creating the sparse matrix
rows = (finaldf2['Node'])
cols = (finaldf2['Outlink'])
n = len(set(rows) | set(cols))
a = np.zeros(n)
a[list(set(cols).difference(set(rows)))] = 1
matrix3 = sparse.csr_matrix((finaldf2.weight,(rows, cols)))

#Creating E,V
e = np.ones((matrix3.shape[0],1))
v = np.ones((matrix3.shape[0],1))/matrix3.shape[0]

In [42]:
#Making sure shapes match
matrix3.shape

(685231, 685231)

In [43]:
finaltest=power_method(matrix3,a,0.85,v,1e-8)

71 iterations


In [44]:
finaltest

array([[1.90811781e-05],
       [4.26600484e-06],
       [2.87615562e-06],
       ...,
       [3.11938602e-07],
       [3.11938602e-07],
       [2.11962172e-03]])

In [45]:
rankdata(finaltest,method='ordinal')[-1]

685220

In [46]:
print('The new node is ranked at',685231-rankdata(finaltest,method='ordinal')[-1],'out of 685231 total nodes')

The new node is ranked at 11 out of 685231 total nodes


In [47]:
pagerankfinal=rankdata(finaltest,method='ordinal')

In [48]:
#10 highest ranking nodes now
tenhighestnow=pagerankfinal.argsort()[::-1][:10]
tenhighestnow

array([288238,      9, 225465,  54130,   1283, 564114,    113,    114,
       288346,  65122], dtype=int64)

In [49]:
#10 lowest ranking nodes now
tenlowestnow=pagerankfinal.argsort()[::-1][-10:]
tenlowestnow

array([4177, 4151, 4125, 3916, 3915, 3914, 3744, 3743, 3742, 3559],
      dtype=int64)

In [50]:
print(tenhighestnow == tenhighest)

[ True  True  True  True  True  True False False False False]


The new node is ranked 11th from the top and it has affected the ranking of the 10 highest nodes.
The previous 10 highest ranked nodes are now pointing(out-link) at our new node and have boosted its page-rank score making it rank 11th.

To be precise, it affected the last 4 out of the 10 highest ranking nodes.

In [51]:
print(tenlowestnow == tenlowest)

[False False False False False False False False False False]


We see that the 10 lowest ranking nodes have completely changed when adding the new node and that is because they were the ones that our new node is pointing at(Out links to).
In other words,our new node which is now important(Ranked 11th) also points at the previous 10 lowest ranking nodes and has therefore boosted their respective scores and they are no longer amongst the lowest ranking nodes.

We see from question 4 that even though 500 hundred webpages were out-linking towards our new node,that placed at a way lower rank than that of when the 10 most important nodes were pointing to it.

A webpage's PageRank score can be improved by having more and more pages link to it.
Naturally, if such pages have a high PageRank score, their contribution will be even greater.

#### In other words,value matters more than volume.(Quality over quantity)